# <font color='darkblue'>Setup</font> 

## <font color='orange'>Packages</font> 

In [1]:
# Packages -----------------------------------------------#

# Data Analysis
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats
from scipy.ndimage import gaussian_filter
import datetime as dt
from scipy.interpolate import griddata

# Plotting
import cartopy.crs as ccrs
from cartopy.feature import LAND
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.pyplot as mpl
import cmocean

# Timing Processes and Progress
import time
from tqdm import tqdm
import itertools

# make sure the figures plot inline rather than at the end
%matplotlib inline

## <font color='orange'>Functions</font> 

In [2]:
# filtering a 2D matrix ---------------------------------------------------------------#
def gaus_filter_nan(img,sigma = 1):
    # import required packages
    from astropy.convolution import convolve
    from astropy.convolution import Gaussian2DKernel
    
    # create a kernal with std = 1
    kernel = Gaussian2DKernel(x_stddev=1)
    
    #convolve that with the 2d matrix
    img_conv = convolve(img, kernel)

    # set original nan points back to nans
    img_conv[np.isnan(img)] = np.nan

    return img_conv

# put coastlines on map and set xy limits ---------------------------------------------#
def add_land(ax,bounds= [35,120,-20,30]):
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
    ax.add_feature(cfeature.LAND,color='gray',zorder = 1)
    ax.background_patch.set_facecolor('k')
    ax.coastlines(resolution='110m',zorder = 2)
    g = ax.gridlines(draw_labels=True,alpha=0)
    g.xlabels_top = False
    g.ylabels_right = False
    g.xlabel_style = {'size': 15}
    g.ylabel_style = {'size': 15}
    g.xformatter = LONGITUDE_FORMATTER
    g.yformatter = LATITUDE_FORMATTER
    ax.axes.axis('tight')
    ax.set_extent(bounds, crs=ccrs.PlateCarree())
    return None 

# mask coastlines ---------------------------------------------------------------------#
def mask_coast(c_lon,c_lat,bounds):
    data=xr.open_dataset('/tigress/GEOCLIM/LRGROUP/shared_data/pco2_flux_coastal_Roobaert/mask_ocean.nc')
    mask_coast=np.array(data.mask_coastal2).astype(int).T
    lat=np.array(data.latitude)
    lon=np.array(data.longitude)

    mask_lon=np.logical_and(lon>bounds[0],lon<bounds[1])
    mask_lat=np.logical_and(lat>bounds[2],lat<bounds[3])

    lon=lon[mask_lon]
    lat=lat[mask_lat]

    mask_coast=mask_coast[mask_lat]
    mask_coast=mask_coast[:,mask_lon]

    lonlon,latlat=np.meshgrid(lon,lat)

    # what are these for? I should probably fix it
    lon_dot=np.array([70,70])
    lat_dot=np.array([10,19.5])

    mask=[]
    for lo,la in zip(c_lon,c_lat):
        if len(lon[lon<=lo])>0 and len(lat[lat>=la])>0 and len(lon[lon>=lo])>0 and len(lat[lat<=la])>0:
            lon_lim=[lon[lon<=lo][-1],lon[lon>=lo][0]]
            lat_lim=[lat[lat<=la][-1],lat[lat>=la][0]]
            mask_lon=np.logical_or(lon==lon_lim[0],lon==lon_lim[1])
            mask_lat=np.logical_or(lat==lat_lim[0],lat==lat_lim[1])
            mask_tmp=mask_coast[mask_lat]
            mask_tmp=mask_tmp[:,mask_lon]
            mask.append(np.mean(mask_tmp)>0)
        else:
            mask.append(False)
    mask=np.array(mask)
    return mask

## <font color='orange'>Read Data</font> 

In [3]:
# WOA
path_WOA = '../data/WOA/'
infn_WOA = 'woa_grid_processed.nc'

ds_WOA = xr.open_dataset(path_WOA + infn_WOA)

In [4]:
plt.pcolormesh(ds_WOA.doxy_grid[0,-1,:,:])
plt.colorbar()

# <font color='darkblue'>Find WOA TCD, OCD, HCD</font> 

In [5]:
TCP = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
OCP = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
HCP = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

TCP_thresh_20 = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
OCP_thresh_50 = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
TCP_thresh_23 = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
OCP_thresh_100 = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
TCP_thresh_25 = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
OCP_thresh_150 = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

# mean temperature and doxy in water column
doxy_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

doxy_TCP_bot_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_TCP_bot_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

doxy_TCP_200_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_TCP_200_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

doxy_10_50_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_10_50_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

doxy_50_200_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_50_200_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

doxy_10_200_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_10_200_mean = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

# minimum oxygen and temp values
doxy_min = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_min = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

# median oxygen and temp values
doxy_median = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)
temp_median = np.full((ds_WOA.month.shape[0],ds_WOA.lat.shape[0],ds_WOA.lon.shape[0]),np.nan)

n = len(ds_WOA.month)
m = len(ds_WOA.lat)
p = len(ds_WOA.lon)

with tqdm(total=n * m * p) as pbar:
    for tt,ii,jj in itertools.product(np.arange(len(ds_WOA.month)),np.arange(len(ds_WOA.lat)),np.arange(len(ds_WOA.lon))):
        temp = np.array(ds_WOA.temp_grid[tt,:,ii,jj])
        doxy = np.array(ds_WOA.doxy_grid[tt,:,ii,jj])
        psal = np.array(ds_WOA.psal_grid[tt,:,ii,jj])
        pres = np.array(ds_WOA.pres_grid[:,ii,jj])
#         pres = np.array(ds_WOA.pres)
#         depth = ds_WOA.depth

        # gradient method -----------------------------------------#

        # only take upper 10-500dbar of data
        ind = (pres>=10) & (pres<=500)
        upper_temp = temp[ind]
        upper_doxy = doxy[ind]
        upper_psal = psal[ind]
        upper_pres = pres[ind]

        # find gradients, min/max gradient inds, and pressure at min inds
        if np.nansum(~np.isnan(upper_temp)) >= 10: 
            temp_grad = np.gradient(upper_temp,upper_pres)
            # note that nanargmin automatically takes the value closest to the surface
            temp_grad_max_ind = np.nanargmax(np.abs(temp_grad))
            TCP[tt,ii,jj] = upper_pres[temp_grad_max_ind]
            
        if np.nansum(~np.isnan(upper_doxy)) >= 10: 
            doxy_grad = np.gradient(upper_doxy,upper_pres)
            doxy_grad_max_ind = np.nanargmax(np.abs(doxy_grad))
            OCP[tt,ii,jj] = upper_pres[doxy_grad_max_ind]
            
        if np.nansum(~np.isnan(upper_psal)) >= 10: 
            psal_grad = np.gradient(upper_psal,upper_pres)
            psal_grad_max_ind = np.nanargmax(np.abs(psal_grad)) 
            HCP[tt,ii,jj] = upper_pres[psal_grad_max_ind]

        # threshold method -----------------------------------------#
        doxy_thresh = 100
        temp_thresh = 23
        
        temp_ind = temp <= temp_thresh
        if len(pres[temp_ind]) > 0: 
            TCP_thresh_23[tt,ii,jj] = pres[temp_ind][0]
            
        doxy_ind = doxy <= doxy_thresh
        if len(pres[doxy_ind]) > 0: 
            OCP_thresh_100[tt,ii,jj] = pres[doxy_ind][0]
        
        doxy_thresh = 50
        temp_thresh = 20
        
        temp_ind = temp <= temp_thresh
        if len(pres[temp_ind]) > 0: 
            TCP_thresh_20[tt,ii,jj] = pres[temp_ind][0]
            
        doxy_ind = doxy <= doxy_thresh
        if len(pres[doxy_ind]) > 0: 
            OCP_thresh_50[tt,ii,jj] = pres[doxy_ind][0]
        
        doxy_thresh = 150
        temp_thresh = 25
        
        temp_ind = temp <= temp_thresh
        if len(pres[temp_ind]) > 0: 
            TCP_thresh_25[tt,ii,jj] = pres[temp_ind][0]
            
        doxy_ind = doxy <= doxy_thresh
        if len(pres[doxy_ind]) > 0: 
            OCP_thresh_150[tt,ii,jj] = pres[doxy_ind][0]

        # find water column averages -------------------------------#
        
        TCPtmp = np.array(TCP[tt,ii,jj])
        OCPtmp = np.array(OCP[tt,ii,jj])
        
        if ~np.isnan(TCPtmp): # if TCP/OCPtmp is a nan it already has less than 10 pts above 500dbar
                
            if TCPtmp < 200:
                # find mean temperature and doxy TCP-200dbar
                ind = (pres>= TCPtmp) & (pres<= 200)
                print(pres[ind])
                temp_TCP_200_mean[tt,ii,jj] = np.nanmean(temp[ind])
                doxy_TCP_200_mean[tt,ii,jj] = np.nanmean(doxy[ind])
    
            # find mean temperature and doxy TCP-bottom
            ind = (pres>= TCPtmp)
            temp_TCP_bot_mean[tt,ii,jj] = np.nanmean(temp[ind])
            doxy_TCP_bot_mean[tt,ii,jj] = np.nanmean(doxy[ind])
                
            # find mean temperature and doxy top-bottom
            temp_mean[tt,ii,jj] = np.nanmean(temp)
            doxy_mean[tt,ii,jj] = np.nanmean(doxy)

            # find mean temperature and doxy 10-50dbar
            ind = (pres>= 10) & (pres<= 50)
            temp_10_50_mean[tt,ii,jj] = np.nanmean(temp[ind])
            doxy_10_50_mean[tt,ii,jj] = np.nanmean(doxy[ind])

            # find mean temperature and doxy 50-200dbar
            ind = (pres>= 50) & (pres<= 200)
            temp_50_200_mean[tt,ii,jj] = np.nanmean(temp[ind])
            doxy_50_200_mean[tt,ii,jj] = np.nanmean(doxy[ind])

            # find mean temperature and doxy 10-200dbar
            ind = (pres>= 10) & (pres<= 200)
            temp_10_200_mean[tt,ii,jj] = np.nanmean(temp[ind])
            doxy_10_200_mean[tt,ii,jj] = np.nanmean(doxy[ind])

            # find min temperature and doxy 
            temp_min[tt,ii,jj] = np.nanmin(temp)
            doxy_min[tt,ii,jj] = np.nanmin(doxy)

            # find min temperature and doxy 
            temp_median[tt,ii,jj] = np.nanmedian(temp)
            doxy_median[tt,ii,jj] = np.nanmedian(doxy)

    
        pbar.update(1)
    

  0%|          | 21/68400 [00:00<05:39, 201.21it/s]

[ 50.36727142  55.41074371  60.44072723  65.48419952  70.51418304
  75.55764771  80.60112     85.63110352  90.67457581  95.7180481
 100.74803162 125.9384079  151.14227295 176.33265686]
[ 50.36727142  55.41074371  60.44072723  65.48419952  70.51418304
  75.55764771  80.60112     85.63110352  90.67457581  95.7180481
 100.74803162 125.9384079  151.14227295 176.33265686]
[ 50.36727142  55.41074371  60.44072723  65.48419952  70.51418304
  75.55764771  80.60112     85.63110352  90.67457581  95.7180481
 100.74803162 125.9384079  151.14227295 176.33265686]
[ 40.29381943  45.3372879   50.36727142  55.41074371  60.44072723
  65.48419952  70.51418304  75.55764771  80.60112     85.63110352
  90.67457581  95.7180481  100.74803162 125.9384079  151.14227295
 176.33265686]
[ 40.29381943  45.3372879   50.36727142  55.41074371  60.44072723
  65.48419952  70.51418304  75.55764771  80.60112     85.63110352
  90.67457581  95.7180481  100.74803162 125.9384079  151.14227295
 176.33265686]
[ 40.29381943  45.3

  0%|          | 63/68400 [00:00<05:37, 202.45it/s]/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in less_equal
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in less_equal
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in less_equal
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: invalid value encountered in less_equal
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in less_equal
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in less_equal
  0%|          | 84/68400 [00:00<05:34, 204.08it/s]

[ 30.22036362  35.26383209  40.29381943  45.3372879   50.36727142
  55.41074371  60.44072723  65.48419952  70.51418304  75.55764771
  80.60112     85.63110352  90.67457581  95.7180481  100.74803162
 125.9384079  151.14227295 176.33265686]
[ 30.22036362  35.26383209  40.29381943  45.3372879   50.36727142
  55.41074371  60.44072723  65.48419952  70.51418304  75.55764771
  80.60112     85.63110352  90.67457581  95.7180481  100.74803162
 125.9384079  151.14227295 176.33265686]
[ 30.22036362  35.26383209  40.29381943  45.3372879   50.36727142
  55.41074371  60.44072723  65.48419952  70.51418304  75.55764771
  80.60112     85.63110352  90.67457581  95.7180481  100.74803162
 125.9384079  151.14227295 176.33265686]
[ 35.26383209  40.29381943  45.3372879   50.36727142  55.41074371
  60.44072723  65.48419952  70.51418304  75.55764771  80.60112
  85.63110352  90.67457581  95.7180481  100.74803162 125.9384079
 151.14227295 176.33265686]
[ 35.26383209  40.29381943  45.3372879   50.36727142  55.4107

  0%|          | 128/68400 [00:00<05:28, 208.13it/s]

...

 18%|█▊        | 11994/68400 [00:54<04:28, 210.37it/s]

[ 45.31031799  50.35378647  55.38376999  60.42724228  65.45722961
  70.48721313  75.5306778   80.56066895  85.59065247  90.63411713
  95.67758942 100.70757294 125.88446808 151.07484436 176.26522827]
[ 45.31031799  50.35378647  55.38376999  60.42724228  65.45722961
  70.48721313  75.5306778   80.56066895  85.59065247  90.63411713
  95.67758942 100.70757294 125.88446808 151.07484436 176.26522827]
[ 55.38376999  60.42724228  65.45722961  70.48721313  75.5306778
  80.56066895  85.59065247  90.63411713  95.67758942 100.70757294
 125.88446808 151.07484436 176.26522827]
[ 60.42724228  65.45722961  70.48721313  75.5306778   80.56066895
  85.59065247  90.63411713  95.67758942 100.70757294 125.88446808
 151.07484436 176.26522827]
[ 60.42724228  65.45722961  70.48721313  75.5306778   80.56066895
  85.59065247  90.63411713  95.67758942 100.70757294 125.88446808
 151.07484436 176.26522827]
[ 60.42724228  65.45722961  70.48721313  75.5306778   80.56066895
  85.59065247  90.63411713  95.67758942 100.

 18%|█▊        | 12013/68400 [00:54<04:17, 218.57it/s]


[ 55.38376999  60.42724228  65.45722961  70.48721313  75.5306778
  80.56066895  85.59065247  90.63411713  95.67758942 100.70757294
 125.88446808 151.07484436 176.26522827]


KeyboardInterrupt: 

In [ ]:
 # Example plot

tt,ii,jj = 1,50,25
temp = np.array(ds_WOA.temp_grid[tt,:,ii,jj])
doxy = np.array(ds_WOA.doxy_grid[tt,:,ii,jj])
psal = np.array(ds_WOA.psal_grid[tt,:,ii,jj])
pres = np.array(ds_WOA.pres_grid[:,ii,jj])
# pres = np.array(ds_WOA.pres)
temp_grad = np.gradient(temp,pres)
doxy_grad = np.gradient(doxy,pres)
psal_grad = np.gradient(psal,pres)

upper_temp = temp[pres<=500]
upper_psal = temp[psal<=500]
upper_pres = pres[pres<=500]

f, (ax1,ax2) = plt.subplots(1, 2)
ax1.plot(temp,-1*pres)
ax2.plot(temp_grad,-1*pres)
ax2.plot(temp_grad,-1*np.full(pres.shape,TCP[tt,ii,jj]))
ax2.plot(temp_grad,-1*np.full(pres.shape,TCP_thresh_23[tt,ii,jj]))
plt.suptitle('Temp')

f, (ax1,ax2) = plt.subplots(1, 2)
ax1.plot(doxy,-1*pres)
ax2.plot(doxy_grad,-1*pres)
ax2.plot(doxy_grad,-1*np.full(pres.shape,OCP[tt,ii,jj]))
ax2.plot(doxy_grad,-1*np.full(pres.shape,OCP_thresh_100[tt,ii,jj]))
plt.suptitle('Doxy')


f, (ax1,ax2) = plt.subplots(1, 2)
ax1.plot(psal,-1*pres)
ax2.plot(psal_grad,-1*pres)
ax2.plot(psal_grad,-1*np.full(pres.shape,HCP[tt,ii,jj]))
plt.suptitle('Sal')

In [ ]:
plt.pcolormesh(np.nanmean(TCP_thresh_23,axis=0))
plt.colorbar()

In [ ]:
# convert to xarray dataset
ds=xr.Dataset(coords={'lon': ds_WOA.lon,
                    'lat': ds_WOA.lat,
                    'month': ds_WOA.month})

# add variables to dataset

ds["TCP"]=xr.DataArray(TCP,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds["OCP"]=xr.DataArray(OCP,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds["HCP"]=xr.DataArray(HCP,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])

ds["TCP_thresh_20"]=xr.DataArray(TCP_thresh_20,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds["OCP_thresh_50"]=xr.DataArray(OCP_thresh_50,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])

ds["TCP_thresh_23"]=xr.DataArray(TCP_thresh_23,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds["OCP_thresh_100"]=xr.DataArray(OCP_thresh_100,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])

ds["TCP_thresh_25"]=xr.DataArray(TCP_thresh_25,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds["OCP_thresh_150"]=xr.DataArray(OCP_thresh_150,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])

ds['temp_mean'] = xr.DataArray(temp_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_mean'] = xr.DataArray(doxy_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['temp_10_200_mean'] = xr.DataArray(temp_10_200_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_10_200_mean'] = xr.DataArray(doxy_10_200_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['temp_10_50_mean'] = xr.DataArray(temp_10_50_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_10_50_mean'] = xr.DataArray(doxy_10_50_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['temp_50_200_mean'] = xr.DataArray(temp_50_200_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_50_200_mean'] = xr.DataArray(doxy_50_200_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['temp_TCP_200_mean'] = xr.DataArray(temp_TCP_200_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_TCP_200_mean'] = xr.DataArray(doxy_TCP_200_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['temp_TCP_bot_mean'] = xr.DataArray(temp_TCP_bot_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_TCP_bot_mean'] = xr.DataArray(doxy_TCP_bot_mean,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])

ds['temp_min'] = xr.DataArray(temp_min,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_min'] = xr.DataArray(doxy_min,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])

ds['temp_median'] = xr.DataArray(temp_median,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
ds['doxy_median'] = xr.DataArray(doxy_median,dims = ['month','lat', 'lon'],
                     coords =[ds_WOA.month,ds_WOA.lat,ds_WOA.lon])
import os

# AWG data
outfn = '../data/CD/cd_woa.nc'

# delete if already present
if os.path.isfile(outfn):
    os.remove(outfn)

ds.to_netcdf(outfn,mode='w',format = "NETCDF4")